In [62]:
from gensim.models import KeyedVectors
# glove_input_file = './util/glove.twitter.27B.200d.txt'
glove_input_file = './util/glove.6B.100d.txt'

# Load GloVe vectors directly into a KeyedVectors instance
model_glove = KeyedVectors.load_word2vec_format(glove_input_file, binary=False, no_header=True)

In [63]:
import fasttext
model_ft = fasttext.load_model('./util/cc.en.300.bin')

In [64]:
from gensim.models import KeyedVectors

# Load the model
filename = './util/GoogleNews-vectors-negative300.bin'
model_wv = KeyedVectors.load_word2vec_format(filename, binary=True)

In [65]:
all_models = [model_glove, model_ft, model_wv]
model = model_glove

In [66]:
from scipy.spatial.distance import cosine

def get_min_word(base, word, model, get_all=False):
    """
    Paramters:
    base (list or iterable)
    word (str)
    """
    
    
    def cosine_similarity(vec1, vec2):
        return 1 - cosine(vec1, vec2)
    
    result = {}
    
    if word in model:
        word_vector = model[word]
    else:
        return None
    
    for v in base:
        if v not in model:
            continue
        base_vector = model[v]
        result[v] = cosine_similarity(base_vector, word_vector)
    
    result = dict(sorted(result.items(),
                         key=lambda x: x[1],
                         reverse=True))
    if get_all:
        return result
    
    max_item = list(result.items())[0]
    return max_item

In [67]:
def ensemble_min_word(base, word, models, get_all=False):
    result = {}
    for model in models:
        print("CHECK", get_min_word(base, word, model))
        item = get_min_word(base, word, model)
        if not item:
            return None
        key, value = item
        if key in result.keys():
            curr_count, curr_value = result[key]
            if curr_value < value:
                new_value = (curr_count + 1, value)
            else:
                new_value = (curr_count + 1, curr_value)
            
            result[key] = new_value
        else:
            result[key] = (1, value)
    
    result = sorted(result.items(),
                    key=lambda item: (-item[1][0],
                                      -item[1][1]))
    result = dict(result)
    if get_all:
        return result
    
    max_key, max_value = list(result.items())[0]
    
    return {max_key: max_value[-1]}

In [68]:
# # Example dictionary
# data = {
#     'key1': (10, 3),
#     'key2': (10, 5),
#     'key3': (5, 8),
#     'key4': (15, 1),
#     'key5': (10, 1)
# }

# # Sorting the dictionary
# sorted_data = sorted(data.items(), key=lambda item: (-item[1][0], -item[1][1]))

# # Converting the sorted items back to a dictionary (if needed)
# sorted_dict = dict(sorted_data)

# print(sorted_dict)


In [69]:
sample = ['comfortable', 'cute', 'super']
aspects = ['color', 'size', 'quality']

sample

['comfortable', 'cute', 'super']

In [70]:
main = {}
for s in sample:
    print(f"Sample: {s}")
    result = ensemble_min_word(aspects, s, all_models)
    print(result)

Sample: comfortable
CHECK ('quality', 0.40961605310440063)
CHECK ('quality', 0.26019683480262756)
CHECK ('quality', 0.17304427921772003)
{'quality': 0.40961605310440063}
Sample: cute
CHECK ('color', 0.3296552002429962)
CHECK ('size', 0.17345145344734192)
CHECK ('color', 0.23953045904636383)
{'color': 0.3296552002429962}
Sample: super
CHECK ('size', 0.34308332204818726)
CHECK ('size', 0.17350566387176514)
CHECK ('size', 0.21845990419387817)
{'size': 0.34308332204818726}


In [71]:
main = {}
for s in sample:
    print(f"Sample: {s}")
    result = get_min_word(aspects, s, model_glove)
    print(result)

main

Sample: comfortable
('quality', 0.40961605310440063)
Sample: cute
('color', 0.3296552002429962)
Sample: super
('size', 0.34308332204818726)


{}

In [72]:
get_min_word(aspects, sample[0], model_glove)

('quality', 0.40961605310440063)

In [73]:
import util.utility as util

print(f"Sample: {sample[0]}")
util.get_nearest_word(sample[0], 
                      aspects, model_glove,
                      threshold=0.0, get_all=True)

Sample: comfortable


{'quality': 0.40961605310440063,
 'size': 0.35894331336021423,
 'color': 0.3211316168308258}

In [74]:
print(f"Sample: {sample[1]}")
util.get_nearest_word(sample[1], 
                      aspects, model_glove,
                      threshold=0.0, get_all=True)

Sample: cute


{'color': 0.3296552002429962,
 'size': 0.2790714204311371,
 'quality': 0.20777395367622375}

In [75]:
print(f"Sample: {sample[2]}")
util.get_nearest_word(sample[2], 
                      aspects, model_glove,
                      threshold=0.0, get_all=True)

Sample: super


{'size': 0.34308332204818726,
 'quality': 0.31452566385269165,
 'color': 0.31028398871421814}

In [76]:
print(f"Sample: {sample}")
util.get_nearest_word(' '.join(sample), 
                      aspects, model_glove,
                      threshold=0.0, get_all=True)

Sample: ['comfortable', 'cute', 'super']


{'size': 0.4331209659576416,
 'color': 0.42173415422439575,
 'quality': 0.4129660427570343}

In [77]:
# Import package
import pandas as pd
import numpy as np

from tqdm import tqdm

In [78]:
df = pd.read_csv("./McDonald_s_Reviews.csv", encoding='latin1')
df = df[['reviewer_id', 'review_time', 'review', 'rating']]
print(df.info())
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33396 entries, 0 to 33395
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   reviewer_id  33396 non-null  int64 
 1   review_time  33396 non-null  object
 2   review       33396 non-null  object
 3   rating       33396 non-null  object
dtypes: int64(1), object(3)
memory usage: 1.0+ MB
None


,reviewer_id,review_time,review,rating
0,1,3 months ago,Why does it look like someone spit on my food?...,1 star
1,2,5 days ago,It'd McDonalds. It is what it is as far as the...,4 stars
2,3,5 days ago,Made a mobile order got to the speaker and che...,1 star
3,4,a month ago,My mc. Crispy chicken sandwich was ï¿½ï¿½ï¿½ï¿...,5 stars
4,5,2 months ago,"I repeat my order 3 times in the drive thru, a...",1 star


In [79]:
df = df.iloc[:100]

# Normalization

In [80]:
import util.normalization as norm

In [81]:
# Tokenization into sentence

df_st = df[['reviewer_id', 'review']].copy()

# Sentence Tokenization
df_st['token_sentence'] = df_st['review'].apply(norm.sentence_tokenize)
df_st = df_st.explode('token_sentence')
df_st['token_sentence'] = df_st['token_sentence'].str.strip()

# Remove non-ASCII
df_st['token_sentence'] = df_st['token_sentence'].apply(norm.remove_non_ascii)

# Expand contractions
df_st['token_sentence'] = df_st['token_sentence'].apply(norm.expand_contractions)

# Remove characters
df_st['token_sentence'] = df_st['token_sentence'].apply(norm.remove_characters, args=(True,))

# Remove enter tab
df_st['token_clean'] = df_st['token_sentence'].apply(norm.remove_enter_tab)

# Lemmatization
df_st['token_clean'] = df_st['token_clean'].apply(norm.lemmatize_text)

In [82]:
print(df_st.info())
df_st.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 546 entries, 0 to 99
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   reviewer_id     546 non-null    int64 
 1   review          546 non-null    object
 2   token_sentence  546 non-null    object
 3   token_clean     546 non-null    object
dtypes: int64(1), object(3)
memory usage: 21.3+ KB
None


,reviewer_id,review,token_sentence,token_clean
0,1,Why does it look like someone spit on my food?...,Why does it look like someone spit on my food,why do it look like someone spit on my food
0,1,Why does it look like someone spit on my food?...,I had a normal transaction,i have a normal transaction
0,1,Why does it look like someone spit on my food?...,everyone was chill and polite,everyone be chill and polite
0,1,Why does it look like someone spit on my food?...,but now i dont want to eat this.,but now i dont want to eat this .
0,1,Why does it look like someone spit on my food?...,Im trying not to think about what this milky w...,im try not to think about what this milky whit...


# Determine Aspect with LDA

In [83]:
import util.model as models
import util.utility as util
from util.utility import get_topics, predict_topic
from util.normalization import preprocess_lda

In [84]:
from sklearn.feature_extraction.text import CountVectorizer

texts = df_st['token_clean'].values

# Vectorize the texts using CountVectorizer
vectorizer = CountVectorizer(tokenizer=lambda doc: doc, lowercase=False)

optimal_model, vectorizer = models.get_opt_lda_model(texts, vectorizer) 

C:\Users\Baha Tegar\anaconda3\Lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [85]:
# Print the topics
def print_topics(model, vectorizer, num_words=10):
    topics = model.components_
    feature_names = vectorizer.get_feature_names_out()
    for idx, topic in enumerate(topics):
        print(f"Topic {idx + 1}:")
        print([feature_names[i] for i in topic.argsort()[:-num_words - 1:-1]])
        print([topic.argsort()[:-num_words - 1: -1]])

print_topics(optimal_model, vectorizer)

Topic 1:
['order', 'location', 'drive', 'hour', 'correct', 'thru', 'sure', 'drink', 'time', 'speaker']
[array([362, 295, 153, 248, 101, 533, 510, 152, 536, 483], dtype=int64)]
Topic 2:
['order', 'time', 'sandwich', 'place', 'chicken', 'drive', 'hour', 'person', 'today', 'next']
[array([362, 536, 441, 385,  68, 153, 248, 377, 539, 344], dtype=int64)]
Topic 3:
['order', 'food', 'minute', 'item', 'meal', 'window', 'regular', 'friendly', 'time', 'small']
[array([362, 201, 327, 264, 316, 572, 422, 209, 536, 470], dtype=int64)]
Topic 4:
['drive', 'food', 'thru', 'fast', 'unprofessional', 'employee', 'sauce', 'open', 'cream', 'experience']
[array([153, 201, 533, 189, 555, 167, 445, 360, 105, 181], dtype=int64)]
Topic 5:
['service', 'customer', 'great', 'line', 'good', 'food', 'staff', 'mcdonalds', 'right', 'nothing']
[array([454, 115, 223, 291, 221, 201, 490, 312, 436, 352], dtype=int64)]
Topic 6:
['manager', 'crew', 'star', 'time', 'night', 'management', 'wait', 'experience', 'attitude', 'cl

In [86]:
dict_topics = get_topics(optimal_model, vectorizer, 0.25)
print(dict_topics)

{0: {'drive', 'order', 'location'}, 1: {'good', 'crispy', 'place', 'time', 'chicken', 'sandwich', 'person', 'worker', 'deluxe', 'next', 'window', 'today'}, 2: {'minute', 'food', 'friendly', 'work', 'item', 'regular', 'small', 'meal', 'wait'}, 3: {'cold', 'unprofessional', 'condescending', 'lane', 'compassion', 'rude', 'night', 'company', 'experience', 'smile', 'piece', 'mcds', 'open', 'fast', 'sauce', 'cream', 'mistake', 'notice', 'drink', 'kind', 'employee', 'someone', 'team', 'ticket'}, 4: {'service', 'customer', 'mcdonalds', 'staff', 'nothing', 'great', 'line', 'right'}, 5: {'manager', 'clean', 'negative', 'shift', 'uber', 'counter', 'review', 'crew', 'evening', 'eats', 'star', 'issue', 'zero', 'management', 'attitude', 'morning'}, 6: {'coffee', 'people', 'nice', 'look', 'second', 'large', 'receipt', 'lady', 'state', 'hand', 'something', 'breakfast', 'wrong'}, 7: {'thank', 'waste', 'late', 'week', 'front', 'money', 'chicfila', 'response', 'mcdonald', 'spanish', 'single', 'short', 'n

In [87]:
predict_topic("Why does it look like someone spit on my food", 
              optimal_model, vectorizer)

7

In [88]:
util.get_nearest_word("Why does it look like someone spit on my food", 
                      list(dict_topics[1]), model,
                      threshold=0.4, get_all=True)

{'good': 0.832912027835846,
 'time': 0.7950315475463867,
 'today': 0.7594949007034302,
 'next': 0.7516953945159912,
 'place': 0.7421676516532898,
 'person': 0.7050454020500183,
 'window': 0.5478033423423767,
 'chicken': 0.43014267086982727,
 'worker': 0.4273696839809418}

In [89]:
def fun(x, get_all=False, k=None):
    cat = predict_topic(x, optimal_model, vectorizer)
    if not cat:
        return set()
    topic = dict_topics[cat]
    result = util.get_nearest_word(x, topic, model,
                                   threshold=0.0, get_all=True)
    if get_all:
        return set(result.keys())
    else:
        if not k:
            return set([list(result.keys())[0]])
        return set(list(result.keys())[:k])

df_st['topic_lda'] = df_st['token_clean'].apply(fun, args=(False, 3))
df_st['cat_topic_lda'] = df_st['token_clean'].apply(predict_topic, args=(optimal_model, vectorizer,))

In [90]:
df_st

,reviewer_id,review,token_sentence,token_clean,topic_lda,cat_topic_lda
0,1,Why does it look like someone spit on my food?...,Why does it look like someone spit on my food,why do it look like someone spit on my food,"{need, ready, little}",7
0,1,Why does it look like someone spit on my food?...,I had a normal transaction,i have a normal transaction,"{full, everyone, different}",8
0,1,Why does it look like someone spit on my food?...,everyone was chill and polite,everyone be chill and polite,"{full, hard, everyone}",8
0,1,Why does it look like someone spit on my food?...,but now i dont want to eat this.,but now i dont want to eat this .,{},0
0,1,Why does it look like someone spit on my food?...,Im trying not to think about what this milky w...,im try not to think about what this milky whit...,"{today, good, time}",1
...,...,...,...,...,...,...
98,99,Allways has the best Fries and Ice cream in th...,Allways has the best Fries and Ice cream in th...,allways have the best fry and ice cream in the...,"{kind, team, night}",3
99,100,Mcdonalds is great but they really need to hir...,Mcdonalds is great,mcdonalds be great,"{great, nothing, right}",4
99,100,Mcdonalds is great but they really need to hir...,they really need to hire people who understand...,they really need to hire people who understand...,{},0
99,100,Mcdonalds is great but they really need to hir...,The people are really nice,the people be really nice,"{people, look, something}",6


In [91]:
df_st['cat_topic_lda'].value_counts()

0    133
6     73
8     59
3     54
2     51
4     50
1     47
7     43
5     36
Name: cat_topic_lda, dtype: int64

# Get Aspect

In [92]:
tqdm.pandas()
df_st['sub_aspect'] = df_st['token_clean'].progress_apply(util.get_aspect_rules)

100%|████████████████████████████████████████████████████████████████████████████████| 546/546 [00:50<00:00, 10.73it/s]


In [93]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

text = ""
topic_lda = df_st['topic_lda'].values
sub_aspect = df_st['sub_aspect'].values
for i in range(df_st.shape[0]):
    text = text + " " + " ".join(list(topic_lda[i])) + " " + " ".join(list(sub_aspect[i]))
    

text = text.strip()

wordcloud = WordCloud(background_color='white').generate(text)
plt.style.use('classic')
plt.figure(figsize=(12,10))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.show()

In [94]:
from scipy.spatial.distance import cosine

from scipy.spatial.distance import cosine

def get_aspect_topic(aspect, topic, model):
    max_value, max_key = 0, None
    for e in topic:
#         temp = get_min_word(aspect, e, model, get_all=True)
        temp = ensemble_min_word(aspect, e, all_models)
        if not temp:
            continue
        max_temp = list(temp.items())[0]
        if max_value < max_temp[1]:
            max_value = max_temp[1]
            max_key = max_temp[0]
    
    print(f"{max_key}: {max_value}")
    return max_key

In [131]:
def ensemble_nearest_word(base, word, models, get_all=False):
    result = {}
    for model in models:
        items = util.get_nearest_word(word, base, model, get_all=True)
        print("CHECK", items)
        if not items:
            return None
        key, value = list(items.items())[0]
        if key in result.keys():
            curr_count, curr_value = result[key]
            if curr_value < value:
                new_value = (curr_count + 1, value)
            else:
                new_value = (curr_count + 1, curr_value)
            
            result[key] = new_value
        else:
            result[key] = (1, value)
    
    result = sorted(result.items(),
                    key=lambda item: (-item[1][0],
                                      -item[1][1]))
    result = dict(result)
    if get_all:
        return result
    
    max_key, max_value = list(result.items())[0]
    
    return max_key

In [156]:
get_min_word(aspects, 'normal transaction', model)

In [158]:
ensemble_min_word(aspects, 'transaction', all_models)

CHECK ('service', 0.42004433274269104)
CHECK ('service', 0.30985063314437866)
CHECK ('service', 0.19500765204429626)


{'service': 0.42004433274269104}

In [132]:
ensemble_nearest_word(aspects, 'normal transaction', all_models)

CHECK {'order': 0.5524408221244812, 'service': 0.5327370762825012, 'food': 0.39506110548973083}
CHECK {'order': 0.32569214701652527, 'service': 0.2845168709754944, 'food': 0.14369182288646698}
CHECK {'service': 0.221317857503891, 'order': 0.20698599517345428, 'food': 0.04546332731842995}


'order'

In [124]:
df_st.head(50)

,reviewer_id,review,token_sentence,token_clean,topic_lda,cat_topic_lda,sub_aspect,aspect,pattern_prediction,roberta_prediction,food_pattern,food_roberta,service_pattern,service_roberta,order_pattern,order_roberta
0,1,Why does it look like someone spit on my food?...,Why does it look like someone spit on my food,why do it look like someone spit on my food,"{need, ready, little}",7,{someone spit},[order],negative,negative,0,0,0,0,-99,-99
0,1,Why does it look like someone spit on my food?...,I had a normal transaction,i have a normal transaction,"{full, everyone, different}",8,{normal transaction},[order],positive,positive,0,0,0,0,1,1
0,1,Why does it look like someone spit on my food?...,everyone was chill and polite,everyone be chill and polite,"{full, hard, everyone}",8,{everyone},[order],negative,positive,0,0,0,0,-99,1
0,1,Why does it look like someone spit on my food?...,but now i dont want to eat this.,but now i dont want to eat this .,{},0,{},[],negative,negative,0,0,0,0,0,0
0,1,Why does it look like someone spit on my food?...,Im trying not to think about what this milky w...,im try not to think about what this milky whit...,"{today, good, time}",1,{clear substance},[order],negative,positive,0,0,0,0,-99,1
0,1,Why does it look like someone spit on my food?...,i d sure am not coming back.,i d sure be not come back .,{},0,{},[],positive,negative,0,0,0,0,0,0
1,2,It'd McDonalds. It is what it is as far as the...,It would McDonalds.,it would mcdonalds .,{},0,{},[],negative,positive,0,0,0,0,0,0
1,2,It'd McDonalds. It is what it is as far as the...,It is what it is as far as the food and atmosp...,it be what it be as far as the food and atmosp...,"{kind, experience, night}",3,"{food, atmosphere}","[food, order]",positive,positive,1,1,0,0,1,1
1,2,It'd McDonalds. It is what it is as far as the...,The staff here does make a difference.,the staff here do make a difference .,"{full, hard, everyone}",8,"{staff, difference}","[service, order]",negative,positive,0,0,-99,1,-99,1
1,2,It'd McDonalds. It is what it is as far as the...,They are all friendly,they be all friendly,"{friendly, work, small}",2,{},[],positive,positive,0,0,0,0,1,1


In [133]:
# aspects = ['food', 'service', 'order' ,'place']
aspects = ['food', 'service', 'order' ,]
# aspects = ['service', 'order', 'place']

def fun1(x):
    temp = set()
    for e in x:
#         aspect = util.get_nearest_word(e, aspects, model)
        aspect = ensemble_nearest_word(aspects, e, all_models)
        if aspect:
            temp.add(aspect)
    temp = list(temp)
    return temp

def fun2(x):
#     result = util.get_nearest_word(" ".join(x), aspects, model)
#     print(util.get_nearest_word(" ".join(x), aspects, model, 0.45, get_all=True))
    result = get_aspect_topic(aspects, x, model)
    if not result:
        return []
    return [result]

tqdm.pandas()
# df_st['sub_aspect'].apply(fun)
# df_st['aspect'] = df_st['sub_aspect'].progress_apply(lambda x: list({util.get_aspect(e, aspects=aspects, model=model) 
#                                                                      for e in x}))
df_st['aspect'] = df_st['sub_aspect'].progress_apply(fun1)
# df_st['aspect'] = df_st['topic_lda'].progress_apply(fun2)
# df_st['aspect'] = [list(set(fun1(i) + fun2(j))) for i,j in zip(df_st.sub_aspect, df_st.topic_lda)]
df_st

  0%|                                                                                          | 0/546 [00:00<?, ?it/s]

CHECK {'order': 0.32384100556373596, 'food': 0.29789572954177856, 'service': 0.2657887935638428}
CHECK {'food': 0.23130962252616882, 'order': 0.1199665442109108, 'service': 0.07792798429727554}
CHECK {'food': 0.13232186436653137, 'order': 0.08214081078767776, 'service': 0.06956950575113297}
CHECK {'order': 0.5524408221244812, 'service': 0.5327370762825012, 'food': 0.39506110548973083}
CHECK {'order': 0.32569214701652527, 'service': 0.2845168709754944, 'food': 0.14369182288646698}
CHECK {'service': 0.221317857503891, 'order': 0.20698599517345428, 'food': 0.04546332731842995}
CHECK {'order': 0.4961599111557007, 'food': 0.46295544505119324, 'service': 0.41436824202537537}
CHECK {'food': 0.19248953461647034, 'service': 0.1490120142698288, 'order': 0.13428974151611328}
CHECK {'food': 0.16934338212013245, 'service': 0.09132780879735947, 'order': 0.08983240276575089}
CHECK {'order': 0.5084662437438965, 'food': 0.45111578702926636, 'service': 0.3410953879356384}
CHECK {'order': 0.2009817808866

 12%|█████████▌                                                                      | 65/546 [00:00<00:03, 158.15it/s]

CHECK {'food': 0, 'service': 0, 'order': 0}
CHECK {'food': 0, 'service': 0, 'order': 0}
CHECK {'service': 0.6570282578468323, 'order': 0.6406651139259338, 'food': 0.5168606042861938}
CHECK {'service': 0.32073768973350525, 'order': 0.25489020347595215, 'food': 0.1816270798444748}
CHECK {'service': 0.2396690398454666, 'order': 0.16098612546920776, 'food': 0.14112307131290436}
CHECK {'service': 0.9040338397026062, 'order': 0.5830984115600586, 'food': 0.4917876422405243}
CHECK {'service': 0.8932532072067261, 'order': 0.3117600977420807, 'food': 0.2906327247619629}
CHECK {'service': 0.813677966594696, 'order': 0.1873496025800705, 'food': 0.18604198098182678}
CHECK {'service': 0.49384593963623047, 'order': 0.47398245334625244, 'food': 0.34797540307044983}
CHECK {'service': 0.29739755392074585, 'order': 0.19431839883327484, 'food': 0.1907348930835724}
CHECK {'service': 0.24666975438594818, 'food': 0.12933756411075592, 'order': 0.05593208223581314}
CHECK {'order': 0.8844780325889587, 'service'

 15%|████████████▏                                                                   | 83/546 [00:00<00:03, 154.00it/s]

...

 91%|███████████████████████████████████████████████████████████████████████▊       | 496/546 [00:02<00:00, 262.05it/s]

CHECK {'order': 0.3662059009075165, 'service': 0.3285713195800781, 'food': 0.2770078778266907}
CHECK {'order': 0.1910683661699295, 'food': 0.14040124416351318, 'service': 0.06150640547275543}
CHECK {'food': 0.16770018637180328, 'service': 0.15259556472301483, 'order': 0.10493260622024536}
CHECK {'service': 0.44671565294265747, 'order': 0.2983717918395996, 'food': 0.2736683785915375}
CHECK {'service': 0.26912644505500793, 'food': 0.11063843965530396, 'order': 0.09314128756523132}
CHECK {'service': 0.09355789422988892, 'food': 0.021046537905931473, 'order': 0.004070426803082228}
CHECK {'order': 0.6317206025123596, 'service': 0.4709797203540802, 'food': 0.4141547381877899}
CHECK {'order': 0.30913564562797546, 'service': 0.15318462252616882, 'food': 0.08689697831869125}
CHECK {'order': 0.17517931759357452, 'food': 0.10466484725475311, 'service': 0.07484189420938492}
CHECK {'service': 0.23141512274742126, 'food': 0.18838311731815338, 'order': 0.1660282015800476}
CHECK {'service': 0.18001998

100%|███████████████████████████████████████████████████████████████████████████████| 546/546 [00:02<00:00, 211.45it/s]


CHECK {'order': 0.6403210163116455, 'service': 0.3656177222728729, 'food': 0.3095211684703827}
CHECK {'order': 0.4431309103965759, 'service': 0.11828316003084183, 'food': 0.009809467010200024}
CHECK {'order': 0.35347825288772583, 'service': 0.018405841663479805, 'food': 0.016348375007510185}
CHECK {'order': 1, 'service': 0.5954257249832153, 'food': 0.4973985254764557}
CHECK {'order': 1, 'service': 0.2689184546470642, 'food': 0.19198539853096008}
CHECK {'order': 1, 'service': 0.14488232135772705, 'food': 0.12748083472251892}
CHECK {'order': 0.6218794584274292, 'service': 0.5757685303688049, 'food': 0.4879383146762848}
CHECK {'order': 0.19987572729587555, 'service': 0.17611487209796906, 'food': 0.10656939446926117}
CHECK {'service': 0.14621096849441528, 'order': 0.10849481076002121, 'food': 0.0888483077287674}
CHECK {'service': 0.5675715208053589, 'order': 0.3750501573085785, 'food': 0.3567757308483124}
CHECK {'service': 0.3520180583000183, 'food': 0.15817801654338837, 'order': 0.0863293

,reviewer_id,review,token_sentence,token_clean,topic_lda,cat_topic_lda,sub_aspect,aspect,pattern_prediction,roberta_prediction,food_pattern,food_roberta,service_pattern,service_roberta,order_pattern,order_roberta
0,1,Why does it look like someone spit on my food?...,Why does it look like someone spit on my food,why do it look like someone spit on my food,"{need, ready, little}",7,{someone spit},[food],negative,negative,0,0,0,0,-99,-99
0,1,Why does it look like someone spit on my food?...,I had a normal transaction,i have a normal transaction,"{full, everyone, different}",8,{normal transaction},[order],positive,positive,0,0,0,0,1,1
0,1,Why does it look like someone spit on my food?...,everyone was chill and polite,everyone be chill and polite,"{full, hard, everyone}",8,{everyone},[food],negative,positive,0,0,0,0,-99,1
0,1,Why does it look like someone spit on my food?...,but now i dont want to eat this.,but now i dont want to eat this .,{},0,{},[],negative,negative,0,0,0,0,0,0
0,1,Why does it look like someone spit on my food?...,Im trying not to think about what this milky w...,im try not to think about what this milky whit...,"{today, good, time}",1,{clear substance},[order],negative,positive,0,0,0,0,-99,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,99,Allways has the best Fries and Ice cream in th...,Allways has the best Fries and Ice cream in th...,allways have the best fry and ice cream in the...,"{kind, team, night}",3,"{ice cream, allways, best fry}","[service, food]",positive,positive,1,1,0,0,1,1
99,100,Mcdonalds is great but they really need to hir...,Mcdonalds is great,mcdonalds be great,"{great, nothing, right}",4,{mcdonalds},[food],positive,positive,1,1,0,0,1,1
99,100,Mcdonalds is great but they really need to hir...,they really need to hire people who understand...,they really need to hire people who understand...,{},0,"{people, english order, english, spanish speaker}","[food, order]",negative,positive,-99,1,0,0,-99,1
99,100,Mcdonalds is great but they really need to hir...,The people are really nice,the people be really nice,"{people, look, something}",6,{people},[food],positive,positive,1,1,0,0,0,0


In [136]:
df_st['aspect'].value_counts()

[]                        142
[order]                   107
[food]                    104
[service]                  72
[food, order]              42
[service, order]           40
[service, food]            13
[food, service]            10
[service, food, order]     10
[food, service, order]      6
Name: aspect, dtype: int64

In [150]:
df_st['sub_aspect'].value_counts()

{}                                                                     140
{order}                                                                 13
{location}                                                               4
{food}                                                                   4
{line}                                                                   4
                                                                      ... 
{pinballz arcade, nothing, line, multiple time, parking lot}             1
{minute, people, dinner food, order, parking lot}                        1
{room temperature, temperature caramel, minute wait, caramel mocha}      1
{sarcasm}                                                                1
{people}                                                                 1
Name: sub_aspect, Length: 367, dtype: int64

# Sentiment Analysis

In [138]:
import util.model as models

In [139]:
sample = df_st['token_clean'].iloc[4]
print(sample)
models.pattern_lexicon_model(sample)

im try not to think about what this milky white or clear substance be all over my food


'negative'

In [140]:
tqdm.pandas()
df_st['pattern_prediction'] = df_st['token_clean'].progress_apply(models.pattern_lexicon_model)

100%|██████████████████████████████████████████████████████████████████████████████| 546/546 [00:00<00:00, 2896.19it/s]


In [141]:
tqdm.pandas()
df_st['roberta_prediction'] = df_st['token_sentence'].progress_apply(models.roberta_model)

100%|████████████████████████████████████████████████████████████████████████████████| 546/546 [14:56<00:00,  1.64s/it]


# Get Result

In [142]:
def get_result(aspect, model):
    
    # Initialization
    get_aspects = df_st['aspect'].values
    get_predictions = df_st[model].values
    
    temp = []

    for i in range(len(get_aspects)):
        if (not get_aspects[i]) or (aspect not in get_aspects[i]):
            temp.append(0)
        else:
            if get_predictions[i] == 'negative':
                temp.append(-99)
            else:
                temp.append(1)
    return np.array(temp)

In [143]:
get_aspects = df_st['aspect'].values
get_pattern = df_st['pattern_prediction'].values
get_roberta = df_st['roberta_prediction'].values

new_fields = set()
for aspect in aspects:
    
    df_st[aspect + f"_pattern"] = get_result(aspect, 'pattern_prediction')
    df_st[aspect + f"_roberta"] = get_result(aspect, 'roberta_prediction')
    new_fields.add(aspect + f"_pattern")
    new_fields.add(aspect + f"_roberta")
new_fields = list(new_fields)
print(df_st.info())
df_st

<class 'pandas.core.frame.DataFrame'>
Int64Index: 546 entries, 0 to 99
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   reviewer_id         546 non-null    int64 
 1   review              546 non-null    object
 2   token_sentence      546 non-null    object
 3   token_clean         546 non-null    object
 4   topic_lda           546 non-null    object
 5   cat_topic_lda       546 non-null    int64 
 6   sub_aspect          546 non-null    object
 7   aspect              546 non-null    object
 8   pattern_prediction  546 non-null    object
 9   roberta_prediction  546 non-null    object
 10  food_pattern        546 non-null    int32 
 11  food_roberta        546 non-null    int32 
 12  service_pattern     546 non-null    int32 
 13  service_roberta     546 non-null    int32 
 14  order_pattern       546 non-null    int32 
 15  order_roberta       546 non-null    int32 
dtypes: int32(6), int64(2), obje

,reviewer_id,review,token_sentence,token_clean,topic_lda,cat_topic_lda,sub_aspect,aspect,pattern_prediction,roberta_prediction,food_pattern,food_roberta,service_pattern,service_roberta,order_pattern,order_roberta
0,1,Why does it look like someone spit on my food?...,Why does it look like someone spit on my food,why do it look like someone spit on my food,"{need, ready, little}",7,{someone spit},[food],negative,negative,-99,-99,0,0,0,0
0,1,Why does it look like someone spit on my food?...,I had a normal transaction,i have a normal transaction,"{full, everyone, different}",8,{normal transaction},[order],positive,positive,0,0,0,0,1,1
0,1,Why does it look like someone spit on my food?...,everyone was chill and polite,everyone be chill and polite,"{full, hard, everyone}",8,{everyone},[food],negative,positive,-99,1,0,0,0,0
0,1,Why does it look like someone spit on my food?...,but now i dont want to eat this.,but now i dont want to eat this .,{},0,{},[],negative,negative,0,0,0,0,0,0
0,1,Why does it look like someone spit on my food?...,Im trying not to think about what this milky w...,im try not to think about what this milky whit...,"{today, good, time}",1,{clear substance},[order],negative,positive,0,0,0,0,-99,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,99,Allways has the best Fries and Ice cream in th...,Allways has the best Fries and Ice cream in th...,allways have the best fry and ice cream in the...,"{kind, team, night}",3,"{ice cream, allways, best fry}","[service, food]",positive,positive,1,1,1,1,0,0
99,100,Mcdonalds is great but they really need to hir...,Mcdonalds is great,mcdonalds be great,"{great, nothing, right}",4,{mcdonalds},[food],positive,positive,1,1,0,0,0,0
99,100,Mcdonalds is great but they really need to hir...,they really need to hire people who understand...,they really need to hire people who understand...,{},0,"{people, english order, english, spanish speaker}","[food, order]",negative,positive,-99,1,0,0,-99,1
99,100,Mcdonalds is great but they really need to hir...,The people are really nice,the people be really nice,"{people, look, something}",6,{people},[food],positive,positive,1,1,0,0,0,0


In [144]:
grouped_df = df_st[ ['reviewer_id'] + new_fields ].groupby('reviewer_id').sum()

grouped_df = grouped_df.applymap(lambda x: 0 if x < 0 else 1).reset_index()

grouped_df

,reviewer_id,service_roberta,order_roberta,food_roberta,food_pattern,order_pattern,service_pattern
0,1,1,1,0,0,0,1
1,2,1,1,1,1,0,0
2,3,0,1,0,0,0,0
3,4,1,1,1,0,1,0
4,5,1,0,1,1,0,1
...,...,...,...,...,...,...,...
95,96,0,0,1,0,0,0
96,97,0,1,0,0,1,0
97,98,0,0,1,1,0,0
98,99,1,1,1,1,1,1


In [145]:
result = df.merge(grouped_df, on='reviewer_id', how='left')

print(result.info())
result.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100 entries, 0 to 99
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   reviewer_id      100 non-null    int64 
 1   review_time      100 non-null    object
 2   review           100 non-null    object
 3   rating           100 non-null    object
 4   service_roberta  100 non-null    int64 
 5   order_roberta    100 non-null    int64 
 6   food_roberta     100 non-null    int64 
 7   food_pattern     100 non-null    int64 
 8   order_pattern    100 non-null    int64 
 9   service_pattern  100 non-null    int64 
dtypes: int64(7), object(3)
memory usage: 8.6+ KB
None


,reviewer_id,review_time,review,rating,service_roberta,order_roberta,food_roberta,food_pattern,order_pattern,service_pattern
0,1,3 months ago,Why does it look like someone spit on my food?...,1 star,1,1,0,0,0,1
1,2,5 days ago,It'd McDonalds. It is what it is as far as the...,4 stars,1,1,1,1,0,0
2,3,5 days ago,Made a mobile order got to the speaker and che...,1 star,0,1,0,0,0,0
3,4,a month ago,My mc. Crispy chicken sandwich was ï¿½ï¿½ï¿½ï¿...,5 stars,1,1,1,0,1,0
4,5,2 months ago,"I repeat my order 3 times in the drive thru, a...",1 star,1,0,1,1,0,1


# Evaluation

In [146]:
df_val = pd.read_csv('validation.csv').dropna()

print(df_val.info())
df_val.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100 entries, 0 to 99
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   reviewer_id  100 non-null    int64 
 1   review_time  100 non-null    object
 2   review       100 non-null    object
 3   rating       100 non-null    object
 4   food         100 non-null    object
 5   service      100 non-null    object
 6   order        100 non-null    object
 7   place        100 non-null    object
dtypes: int64(1), object(7)
memory usage: 7.0+ KB
None


,reviewer_id,review_time,review,rating,food,service,order,place
0,1,3 months ago,Why does it look like someone spit on my food?...,1 star,Negative,Neutral,Positive,Neutral
1,2,5 days ago,It'd McDonalds. It is what it is as far as the...,4 stars,Neutral,Positive,Neutral,Positive
2,3,5 days ago,Made a mobile order got to the speaker and che...,1 star,Neutral,Negative,Negative,Neutral
3,4,a month ago,My mc. Crispy chicken sandwich was ï¿½ï¿½ï¿½ï¿...,5 stars,Positive,Positive,Neutral,Neutral
4,5,2 months ago,"I repeat my order 3 times in the drive thru, a...",1 star,Negative,Negative,Negative,Neutral


In [147]:
def mapping_sentiment(x):
    if x == 'Negative':
        return 0
    else:
        return 1    

def join_aspect(x, y):
    if x==1 and y==1:
        return 1
    else:
        return 0
    
for aspect in ['food', 'service', 'order', 'place']:
    df_val[aspect] = df_val[aspect].apply(mapping_sentiment)

# df_val['service'] = [join_aspect(x, y) for x, y in zip(df_val.service, df_val.order)]
# df_val = df_val.drop('order', axis=1)
print(df_val.info())
df_val.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100 entries, 0 to 99
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   reviewer_id  100 non-null    int64 
 1   review_time  100 non-null    object
 2   review       100 non-null    object
 3   rating       100 non-null    object
 4   food         100 non-null    int64 
 5   service      100 non-null    int64 
 6   order        100 non-null    int64 
 7   place        100 non-null    int64 
dtypes: int64(5), object(3)
memory usage: 7.0+ KB
None


,reviewer_id,review_time,review,rating,food,service,order,place
0,1,3 months ago,Why does it look like someone spit on my food?...,1 star,0,1,1,1
1,2,5 days ago,It'd McDonalds. It is what it is as far as the...,4 stars,1,1,1,1
2,3,5 days ago,Made a mobile order got to the speaker and che...,1 star,1,0,0,1
3,4,a month ago,My mc. Crispy chicken sandwich was ï¿½ï¿½ï¿½ï¿...,5 stars,1,1,1,1
4,5,2 months ago,"I repeat my order 3 times in the drive thru, a...",1 star,0,0,0,1


In [148]:
from sklearn.metrics import classification_report, roc_auc_score

print("Prediction pattern")
print("==============================================================")
for aspect in aspects:
    try:
        actual = df_val[aspect]
        pred = result[aspect + "_pattern"]
        print(f"ROC AUC Score: ", roc_auc_score(actual, pred))
        print(f"Result {aspect}")
        print(classification_report(actual, pred))
    except:
        continue

Prediction pattern
ROC AUC Score:  0.652875175315568
Result food
              precision    recall  f1-score   support

           0       0.41      0.87      0.56        31
           1       0.88      0.43      0.58        69

    accuracy                           0.57       100
   macro avg       0.65      0.65      0.57       100
weighted avg       0.74      0.57      0.57       100

ROC AUC Score:  0.6175824175824175
Result service
              precision    recall  f1-score   support

           0       0.74      0.69      0.71        65
           1       0.49      0.54      0.51        35

    accuracy                           0.64       100
   macro avg       0.61      0.62      0.61       100
weighted avg       0.65      0.64      0.64       100

ROC AUC Score:  0.6621787025703795
Result order
              precision    recall  f1-score   support

           0       0.69      0.79      0.74        57
           1       0.66      0.53      0.59        43

    accuracy       

In [149]:
print("Prediction roberta")
print("==============================================================")
for aspect in aspects:
    try:
        actual = df_val[aspect]
        pred = result[aspect + "_roberta"]
        print(f"ROC AUC Score: ", roc_auc_score(actual, pred))
        print(f"Result {aspect}")
        print(classification_report(actual, pred))
    except:
        continue

Prediction roberta
ROC AUC Score:  0.7059373539036933
Result food
              precision    recall  f1-score   support

           0       0.49      0.77      0.60        31
           1       0.86      0.64      0.73        69

    accuracy                           0.68       100
   macro avg       0.68      0.71      0.67       100
weighted avg       0.75      0.68      0.69       100

ROC AUC Score:  0.7659340659340659
Result service
              precision    recall  f1-score   support

           0       0.91      0.65      0.76        65
           1       0.57      0.89      0.70        35

    accuracy                           0.73       100
   macro avg       0.74      0.77      0.73       100
weighted avg       0.79      0.73      0.74       100

ROC AUC Score:  0.7609139126886986
Result order
              precision    recall  f1-score   support

           0       0.81      0.75      0.78        57
           1       0.70      0.77      0.73        43

    accuracy      